In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the numpy, pandas and matplot packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows',150)

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

<h1>Import and Read</h1>
<b>Import all the dataset and store it in a variable previous, application, description.<b>

In [ ]:
#import the dataset
previous= pd.read_csv('/content/drive/MyDrive/Stige/Loan Case Study/previous_application.csv')
application= pd.read_csv('/content/drive/MyDrive/Stige/Loan Case Study/application_data.csv')
description= pd.read_csv('/content/drive/MyDrive/Stige/Loan Case Study/columns_description.csv',encoding='cp1252')

FileNotFoundError: ignored

<b>Inspect the dataset find out null values and percentage of null values column wise.</b>

In [ ]:
#column-wise null values
column_wise= application.isnull().sum(axis=0)
print(column_wise)

<b>column-wise null percentage values.</b>

In [ ]:
#column-wise null percentage values
columns = application.isnull()
column_wise_per=round(columns.sum()/len(application.index) *100 ,2)
print(column_wise_per)

**Find out all the columns having null values percenatage greater then 40.**

In [ ]:
#columns having null values percentage greater then 40
more = column_wise_per[column_wise_per > 40]
print(more)

<b>Count of columns having null percentage greater then 40.</b>

In [ ]:
print("No of columns having null value percenatge greater then 40 is : ",more.count())

<b>drop all the columns having null values percentage greater then 40.</b>

In [ ]:
#drop all the columns having null values percentage greater then 40
application.dropna(thresh=len(application) * .6, axis='columns',inplace=True)

<b>Inscpect dataset after dropping values.</b>

In [ ]:
#columns with more then 60% not null
application.info()
application.shape

In [ ]:
application

<b>Drop all the columns which are not usefull for analysis.</b>

In [ ]:
not_usefull=['EXT_SOURCE_2',	'EXT_SOURCE_3','FLAG_DOCUMENT_2',	'FLAG_DOCUMENT_3'	,'FLAG_DOCUMENT_4',	'FLAG_DOCUMENT_5',	'FLAG_DOCUMENT_6',	'FLAG_DOCUMENT_7'	,'FLAG_DOCUMENT_8',	'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10',	'FLAG_DOCUMENT_11',	'FLAG_DOCUMENT_12',	'FLAG_DOCUMENT_13',	'FLAG_DOCUMENT_14',	'FLAG_DOCUMENT_15',	'FLAG_DOCUMENT_16',	'FLAG_DOCUMENT_17',	'FLAG_DOCUMENT_18',
             'FLAG_DOCUMENT_19',	'FLAG_DOCUMENT_20',	'FLAG_DOCUMENT_21']

In [ ]:
application.drop(labels=not_usefull, axis='columns',inplace=True)

<b>Find the columns with less null and try to fill them.
We have very few XNA values in gender replace then with max gender i.e. F .</b>


In [ ]:
application['CODE_GENDER'].value_counts()
#as max no of customer we can replace XNA by female

In [ ]:
application['CODE_GENDER'].replace({'XNA':'F'},inplace=True) #replace XNA by F 

<b>AMT_ANNUITY also have very few null values. fill the null values with the median.</b>

In [ ]:
application['AMT_ANNUITY'].isnull().sum()/len(application)*100

In [ ]:
#fill up values of AMT_ANNUITY has the no of null values are low 
mean_AMT_ANNUITY = application['AMT_ANNUITY'].mean()
application['AMT_ANNUITY'].fillna(value = mean_AMT_ANNUITY,inplace=True)

<b>As Organization_type is one of the important aspect considered while providing the loan. XNA values are droped.</b>

In [ ]:
application['ORGANIZATION_TYPE'].value_counts()

In [ ]:
application.drop(application[application['ORGANIZATION_TYPE'] == 'XNA'].index, inplace = True)

In [ ]:
previous.drop(previous[previous['NAME_CASH_LOAN_PURPOSE'] == 'XNA'].index, inplace = True)
previous.drop(previous[previous['NAME_CASH_LOAN_PURPOSE'] == 'XAP'].index, inplace = True)
previous.drop(previous[previous['NAME_CASH_LOAN_PURPOSE'] == 'Other'].index, inplace = True)

<h2>Identify Outliers in the dataset.</h2>

In [ ]:
#'AMT_INCOME_TOTAL' 
features = ['CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY']
plt.figure(figsize = (9,7), dpi=300)
for i in enumerate(features):
    plt.subplot(2, 2, i[0]+1)
    sns.boxplot(x = i[1], data = application )
plt.show()

<b>CNT_CHILDREN:-</b> The median is probably identical to the first quartile, which is why they overlap. This tends to happen when you have a large proportion of identical, low values in the dataset.This means that all the data in between too must have the same value.

<b>AMT_INCOME_TOTAL: -</b> It is the total income of the customer.There is single high value data point as outlier present in AMT_INCOME_TOTAL and Removal this point will drastically impact the box plot for further analysis.

<b>AMT_CREDIT: -</b> It is the amount of loan taken the customer ealier. 1st quartile is slimer as complared to 3rd quartile.It indicates max of amount credit values are greater then meadian

<b>AMT_ANNUITY: -</b> 1st quartile is slimer as complared to 3rd quartile.It indicates max of amount credit values are greater then meadian

<h2>Bin Creation</h2>
<b>Bin creation for values in a continous order.</b>

In [ ]:
#create bin for income range of customers
bins = [0,100000,200000,300000,400000,500000,1500000,2500000,5000000 ,10000000000]
slot = ['<100000', '100000-200000','200000-300000','300000-400000','400000-500000', '500000-1500000','1500000-2500000','2500000-5000000','5000000-10000000000']

application['AMT_INCOME_RANGE']=pd.cut(application['AMT_INCOME_TOTAL'],bins,labels=slot)

In [ ]:
bins = [0,100000,200000,300000,400000,500000,600000,700000,800000,900000,1500000,2500000,5000000 ,10000000000]
slot = ['<100000', '100000-200000','200000-300000','300000-400000','400000-500000', '500000-600000',
        '600000-700000','700000-800000','850000-900000','900000-1500000', '1500000-2500000','2500000-5000000','5000000-10000000000']

application['AMT_CREDIT_RANGE']=pd.cut(application['AMT_CREDIT'],bins,labels=slot)

<h2>Imbalance in the data</h2>
<b>Find uneven distribution.</b>

In [ ]:
priority1 = application.loc[application['TARGET'] == 0]
priority2 = application.loc[application['TARGET'] == 1]
print('People paid their loan on time',len(priority1))
print('People either missed insatllments or were unable to pay on time',len(priority2))

In [ ]:
imbalance = round(len(priority2) / len(priority1) *100 ,2)
print("Imbalance percentage of customers",imbalance)

<h2>Varient analysis</h2>

In [ ]:
priority1 = application.loc[application['TARGET'] == 0]
priority2 = application.loc[application['TARGET'] == 1]
print('People paid their loan on time',len(priority1))
print('People either missed insatllments or were unable to pay on time',len(priority2))

<h2>Univarient Data Analysis</h2>
<b>Categorical and numerical data analysis.*italicized text*</b>

In [ ]:
#categorical Univarient
category = ['AMT_INCOME_RANGE', 'AMT_CREDIT_RANGE','NAME_INCOME_TYPE','NAME_CONTRACT_TYPE']
plt.figure(figsize = (20, 15))

for i in enumerate(category):
    plt.subplot(2, 2, i[0]+1)
    plt.subplots_adjust(hspace=0.5)
    sns.countplot(x = i[1], hue = 'TARGET', data = application)
    plt.rcParams['axes.titlesize'] = 16
    plt.xticks(rotation = 45)
    plt.yscale('log')

<b>AMT_INCOME_RANGE: -</b> Indcates income of the customers.<br>
- Maximum amount of customers who applied for the loan have income in range of 100000-200000.
- Maximum amount if customer who applied for the loan and were unable to pay back belong to the income range of 100000-200000.
- Customers belong to income range of 2500000 5000000 have the lowest no of people who were unable to pay back the loan. 

<b>AMT_CREDIT_RANGE: - </b>Amount of loan taken the customer taken previously.
- Most of the customers applied for the loan amount in the range of 900000-1500000 
- People unable to pay back loan is almost equal in all the diffrent ranges.

<b>NAME_INCOME_TYPE: -</b>Source of the income of the customer applying for the loan.
- customer belonging to category student, Pensioner & Businessman have the payed back the loan on time.
- Most of the customer applied for the maternity or working loan were unable to pay back.

<b>NAME_CONTRACT_TYPE: - </b>Contract product type of the previous application.
- Most of the customers how applied for the cash loan were unable to pay back.



In [ ]:
#numerical univarient continous
features = ['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_LAST_PHONE_CHANGE','DAYS_ID_PUBLISH']
plt.figure(figsize = (20,15))

for i in enumerate(features):
    plt.subplot(2, 2, i[0]+1)
    plt.subplots_adjust(hspace=0.5)
    sns.boxplot(x = 'TARGET', y = i[1], data = application)

**Insights**
- DAYS_BIRTH and DAYS_ID_PUBLISH have lesser outlier <br>
- DAYS_BIRTH The people having lower age are having higher probability of repayment.<br>
- In DAYS_ID_PUBLISH: people changing ID in recent days are relativelty prone to be default.

<h2>Bivarient Data Analysis</h2>

In [ ]:
#bivarient analysis
plt.figure(figsize=(16,12))
plt.xticks(rotation=45)
sns.boxplot(data =priority1, x='NAME_EDUCATION_TYPE',y='AMT_CREDIT', hue ='NAME_FAMILY_STATUS',orient='v')
plt.title('Credit amount vs Education Status')
plt.show()

**Insights**
- Most of the customer applied for the loan belong to Academic degree education category followed by Higher Education, Incomplete higher.
- Higher Education and Secondary education has higher outlier as compared to other categories.

In [ ]:
plt.figure(figsize=(16,12))
plt.xticks(rotation=45)
plt.yscale('log')
sns.boxplot(data = priority1, x='NAME_EDUCATION_TYPE',y='AMT_INCOME_TOTAL', hue ='NAME_FAMILY_STATUS',orient='v')
plt.title('Income amount vs Education Status')
plt.show()

**Insights**
- Customer belonging to academic degree education type have the highest income with lowest outliers.<br>
- Lower secondary education type customers have the lowest income.<br>
- Higher education customer have almost equal income independent of the family status.


<h2>Top 10 correlation for the Client.</h2>

In [ ]:
corr = priority1.corr()
corrdf = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
corrdf = corrdf.unstack().reset_index()
corrdf.columns = ['Var1', 'Var2', 'Correlation']
corrdf.dropna(subset = ['Correlation'], inplace = True)
corrdf['Correlation'] = round(corrdf['Correlation'], 2)
corrdf['Correlation'] = abs(corrdf['Correlation'])
corrdf.sort_values(by = 'Correlation', ascending = False).head(10)

In [ ]:
corr = priority2.corr()
corrdf = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
corrdf = corrdf.unstack().reset_index()
corrdf.columns = ['Var1', 'Var2', 'Correlation']
corrdf.dropna(subset = ['Correlation'], inplace = True)
corrdf['Correlation'] = round(corrdf['Correlation'], 2)
corrdf['Correlation'] = abs(corrdf['Correlation'])
corrdf.sort_values(by = 'Correlation', ascending = False).head(10)

<h2>Important results</h2>

<h4>Proportion of defaulters based on education type</h4>

In [ ]:
#plt.figure(figsize=(10,7),dpi = 300)
plt.xticks(rotation=90)
plt.title("Distribution of NAME_EDUCATION_TYPE for pays on time & defaulter.")
ax = sns.countplot(data = application, x='NAME_EDUCATION_TYPE',order=application['NAME_EDUCATION_TYPE'].value_counts().index,hue = 'TARGET') 

<b>Insights </b>
- Higher the education less the default.

<h4>Proportion of defaulters based on <b> Income Type </b> </h4>

In [ ]:
plt.title("Distribution of NAME_INCOME_TYPE for pays on time & defaulter.")
plt.xticks(rotation=90)
ax = sns.countplot(data = application, x='NAME_INCOME_TYPE',order=application['NAME_INCOME_TYPE'].value_counts().index,hue = 'TARGET') 

<b> Insights</b>
- Most the customer who applied for the loan belong to working income type.
- Student, Pensioner, Businessman must give more loan has they have maximum probability paying back on time
- Better returns of loan as income increases.



<h4>Proportion of defaulters based on <b> Family Status </b> </h4>

In [ ]:
plt.title("Distribution of NAME_FAMILY_STATUS for pays on time & defaulter.")
plt.xticks(rotation=90)
ax = sns.countplot(data = application, x='NAME_FAMILY_STATUS',order=application['NAME_FAMILY_STATUS'].value_counts().index,hue = 'TARGET') 

<b>Insights</b>
- Married people have less possiblity of not able to pay back loans.
- Single people default more.

<h4>Proportion of defaulters based on <b> Previous amount credit and housing type. </b> </h4>

In [ ]:
plt.xticks(rotation=90)
sns.barplot(data =application, y='AMT_CREDIT',hue='TARGET',x='NAME_HOUSING_TYPE')
plt.title('Prev Credit amount vs Housing type')
plt.show()

<b>Insights</b>
- Customer having Name_housing_type as with parents have paid the loan on time followed by House/apartment and Municipal Apartment.
- Most of Co apartment customers were unable to pay back the loan.


<h4>Distribution of <b>purpose </b>based on target</h4>

In [ ]:
plt.figure(figsize=(7,5),dpi = 300)
plt.title('Distribution of purposes with target ')
ax = sns.countplot(data = previous, y= 'NAME_CASH_LOAN_PURPOSE', 
                   order= previous['NAME_CASH_LOAN_PURPOSE'].value_counts().index,hue = 'NAME_CONTRACT_STATUS') 

**Insights**
- Maximum difference for approval and rejection was found for buying a new car.
- Maximum Refused & Approved loan belongs to repair purpose.

<h2> Conclusion</h2>

- The proportion of defaulter is 9.48%.
- Maximum customer applied & approved for the loan were females.
- Most of the cash loans are in default so bank must provide more resolving loans.
- Higher the education less the default.
- Higher the income lesser the default.
- Customer having Name_housing_type as with parents have less default.
- Most of Co apartment housing type customers are defaulters.
- Giving loans to married customer is safer as it has lower default rate.
- Maximum difference for approval and rejection was found for buying a new car.
- Maximum Refused & Approved loan belongs to repair purpose.
